In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

/opt/conda/lib/python3.7/site-packages/fastbook/__init__.py:22: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
from fastbook import * 

In [3]:
import pandas as pd
import re

In [26]:
sample_sub = pd.read_csv("data/sample_submission_stg2.csv"); sample_sub

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
1,img_00007.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
2,img_00009.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
3,img_00018.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
4,img_00027.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
...,...,...,...,...,...,...,...,...,...
13148,test_stg2/image_12149.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
13149,test_stg2/image_12150.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
13150,test_stg2/image_12151.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
13151,test_stg2/image_12152.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283


In [5]:
sample_sub.columns

Index(['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'], dtype='object')

# Load Data and Model

In [33]:
dls = ImageDataLoaders.from_folder(path = "data/train",
                                  valid_pct = 0.2, 
                                  seed = 1991,
                                  item_tfms = Resize(448),
                                  batch_tfms = aug_transforms(),
                                  bs = 64)

In [48]:
dls.vocab

(#8) ['ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT']

In [34]:
learn = cnn_learner(dls, resnet34, metrics = accuracy)

In [35]:
learn.load("resnet34_stage2")

# Create Submission File

In [36]:
test_items1 = get_image_files(Path("data/test_stg1"))
test_items2 = get_image_files(Path("data/test_stg2"))

In [37]:
test_dl1 = dls.test_dl(test_items1)
test_dl2 = dls.test_dl(test_items2)

In [38]:
preds1,_ = learn.get_preds(dl=test_dl1)
preds2,_ = learn.get_preds(dl=test_dl2)

In [40]:
# ['SHARK','OTHER','DOL','LAG', 'NoF']

In [41]:
columns = sample_sub.columns
# columns = columns.drop(["image", 'ALB', 'BET', 'YFT']); columns
columns = columns.drop(["image"]); columns

Index(['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'], dtype='object')

In [43]:
df1 = pd.DataFrame(data = preds1, columns = columns)
df2 = pd.DataFrame(data = preds2, columns = columns)

In [44]:
def get_file_name(path):
    match = re.match('^.*\/([^\/]+\.jpg)', str(path))
    return match.group(1)

In [23]:
test_items1_names = [get_file_name(path) for path in test_items1]
test_items2_names = ["test_stg2/" + get_file_name(path) for path in test_items2]

In [45]:
df1.insert(loc = 0, column = "image", value = test_items1_names)
df2.insert(loc = 0, column = "image", value = test_items2_names)

In [49]:
submission_df = pd.concat([df1, df2]); submission_df

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00950.jpg,0.091092,0.000154,0.000235,0.000069,0.906459,0.001500,0.000114,0.000377
1,img_01384.jpg,0.350268,0.005865,0.010577,0.007593,0.555454,0.038866,0.000931,0.030448
2,img_03252.jpg,0.999649,0.000058,0.000004,0.000014,0.000052,0.000026,0.000019,0.000179
3,img_00138.jpg,0.992160,0.000052,0.000210,0.000158,0.005971,0.000221,0.000442,0.000787
4,img_04073.jpg,0.015887,0.024084,0.000937,0.006072,0.002245,0.007861,0.000374,0.942539
...,...,...,...,...,...,...,...,...,...
12148,test_stg2/image_06524.jpg,0.995513,0.000817,0.000013,0.000012,0.000286,0.000121,0.000009,0.003229
12149,test_stg2/image_02773.jpg,0.329281,0.166713,0.059739,0.023515,0.000922,0.002535,0.004770,0.412524
12150,test_stg2/image_05407.jpg,0.943119,0.005028,0.000246,0.000086,0.000043,0.000111,0.000051,0.051316
12151,test_stg2/image_00875.jpg,0.968230,0.000887,0.001467,0.001632,0.012477,0.000582,0.000894,0.013831


In [55]:
file_path = "data/initial_submission.csv"

In [56]:
submission_df.to_csv(file_path, index = False)

# Submitting to Kaggle

In [54]:
competition =  "the-nature-conservancy-fisheries-monitoring"

In [57]:
!kaggle competitions submit the-nature-conservancy-fisheries-monitoring -f $file_path -m"Initial submission" 

100%|███████████████████████████████████████| 1.53M/1.53M [00:02<00:00, 647kB/s]
Successfully submitted to The Nature Conservancy Fisheries Monitoring